# Chaining Prompts 
<https://www.bilibili.com/video/BV1rk4y1W7hj/>

In [34]:
import os
import openai
import utils
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
# print (openai.api_key)

In [40]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0.6, max_tokens=2800):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    ### token_dict = {'prompt_tokens': response['usage']['prompt_tokens']}
    return response.choices[0].message["content"]


In [36]:
delimiter = "####"
system_message = f""" 
You will be provided with customer service queries. \
The customer service query will be delimited with {delimiter} characters.\
Output a python list of objects, where each object has the following format:\
    'category': <one of Computers and Laptops,Smartphones and Accessories, \
    Televisions and Home Theater Systems,Gaming Consoles and Accessories, \
    Audio Equipment，Cameras and Camcorders>,
OR
    'products': <a list of products that must be found in the allowed products below>
    
Where the categories and products must be found in the customer service query.
If a product is mentioned, it must be associated with the correct category in the allowed products list below.
If no products or categories are found, output an empty list.


Allowed products:

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
Smartx ProPhone
MobiTech PowerCase
Smartx MiniPhone
MobiTech Wireless Charger
Smartx EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere x
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Only output the list of objects, with nothing else.
"""
user_message_1 = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one.\
Also tell me about your tvs """
messages = [
{'role': 'system',
'content': system_message},
{'role' : 'user',
'content': f"{delimiter}{user_message_1}{delimiter}"},
]

category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[{'category': 'Smartphones and Accessories', 'products': ['Smartx ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems'}]


In [1]:
# product information
# products ={
# "TechPro ultrabook":{
#     "name":"TechPro Ultrabook",
#     "category": "Computers and Laptops",
#     "brand":"TechPro",
#     "model_number":"TP-UB100","warranty":"1 year","rating": 4.5,"features":["13.3-inch display","8GB RAM","256GB"],
#     "description":"A sleek and lightweight ultrabook",
#     "price": 799.99
#     },
# "BlueWave Gaming Laptop":{
#     "name":"BlueWave Gaming Laptop",
#     "category":"Computers and Laptops",
#     "brand":"BlueWave",
#     "model_number":"BW-GL200",
#     "warranty":"2 years",
#     "rating": 4.7,
#     "features": ["15.6-inch display","16GB RAM","512G"],
#     "description":"A high-performance gaming laptop ",
#     "price": 1199.99
#     },
# }

In [37]:
import json

def get_product_by_name(name):
    return products.get(name, None)
def get_products_by_category(category):
    return [product for product in products.values() if product["category"]==category]

with open('./product-data.json') as file:
    products = json.load(file)
#     print(products)
#     print(type(products))
    print(get_product_by_name("TechPro Ultrabook"))
    print(get_products_by_category("Computers and Laptops"))

{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB'], 'description': 'A sleek and powerful ultrabook with a 13.3-inch display, 8GB RAM and 256GB of storage.', 'price': 799.99}
[{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB'], 'description': 'A sleek and powerful ultrabook with a 13.3-inch display, 8GB RAM and 256GB of storage.', 'price': 799.99}, {'name': 'BlueWave Gaming Laptop', 'category': 'Computers and Laptops', 'brand': 'BlueWave', 'model_number': 'BW-GL100', 'warranty': '2 years', 'rating': 4.7, 'features': ['15.6-inch display', '16GB RAM', '512GB SSD'], 'description': 'A powerful gaming laptop with a 15.6-inch display, 16GB RAM and 512GB SSD.', 'price': 999.99}, {'name': '

In [38]:
import json

def read_string_to_list(input_string):
    if input_string is None:
        return None
    try:
        input_string = input_string.replace("'", "\"")
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None


category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)
print(type(category_and_product_list))

[{'category': 'Smartphones and Accessories', 'products': ['Smartx ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems'}]
<class 'list'>


In [33]:
def generate_output_string(data_list):
    output_string = ""
    if data_list is None:
        return output_string
    
    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent="\t")
                    else:
                        print(f"Error: Product '{product}'")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent="\t")
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")
            
    return output_string

product_infomation_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_infomation_for_user_message_1)

Error: Product 'None'
{
	"name": "Smartx ProPhone",
	"category": "Smartphones and Accessories",
	"brand": "Smartx",
	"model_number": "SM-PP100",
	"warranty": "2 years",
	"rating": 4.8,
	"features": [
		"6.5-inch OLED display",
		"Triple Camera Array",
		"256GB Storage"
	],
	"description": "A flagship smartphone with a 6.5-inch OLED display, Triple Camera Array and 256GB of storage.",
	"price": 1299.99
}{
	"name": "MobiTech PowerCase",
	"category": "Smartphones and Accessories",
	"brand": "MobiTech",
	"model_number": "MT-PC100",
	"warranty": "1 year",
	"rating": 4.6,
	"features": [
		"10500mAh Battery",
		"Wireless Charging",
		"Qi Compatible"
	],
	"description": "A powerful battery case with a 10500mAh battery, wireless charging and Qi compatibility.",
	"price": 79.99
}{
	"name": "Smartx MiniPhone",
	"category": "Smartphones and Accessories",
	"brand": "Smartx",
	"model_number": "SM-MP100",
	"warranty": "1 year",
	"rating": 4.2,
	"features": [
		"4.5-inch Display",
		"2GB RAM",
		"32GB

In [41]:
system_message="""
You are a customer serive assistant for a large electronic store.\
Respond in a friendly and helpful tone, with very concise answer.\
Make sure to ask the user relevant follow up questions.
"""
user_message_1 = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one.\
Also tell me about your tvs """
messages = [
{'role': 'system',
'content': system_message},
{'role' : 'user',
'content': user_message_1},
{'role' : 'assistant',
'content': f"""Relevant product information:\n{product_infomation_for_user_message_1}"""},    
]

final_response = get_completion_from_messages(messages)
print(final_response)

The Smartx ProPhone is a flagship smartphone with a 6.5-inch OLED display, a triple camera array, and 256GB of storage. It is priced at $1299.99 and comes with a 2-year warranty. 

The Fotosnap DSLR camera is not mentioned in the provided information. Could you please provide more details about the camera you are referring to?

Regarding our TVs, we have a range of options available. Could you please specify which TV model or features you are interested in?
